In [1]:
import cv2

In [2]:
# import cv2
# webcam = cv2.VideoCapture(0)
# while(True):
#     ret,frame = webcam.read()
#     gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
#     cv2.imshow("Frame",gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
#     elif cv2.waitKey(1) & 0xFF == ord('a'):
#         cv2.imwrite('hey.jpg',gray)
# webcam.release()        
# cv2.destroyAllWindows()

In [3]:
import numpy as np
import time
import cv2
import math

In [4]:
labelsPath = "./coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

In [5]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

In [6]:
weightsPath = "./yolov3.weights"
configPath = "./yolov3.cfg"
cap = cv2.VideoCapture('./videos/test_video.mp4')
hasFrame, frame = cap.read()
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))


In [7]:
while cv2.waitKey(1) < 0:
    
    ret,image=cap.read()
    image=cv2.resize(image,(640,360))
    (H, W) = image.shape[:2]
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    blob = cv2.dnn.blobFromImage(image, 1 / 300.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    print("Frame Prediction Time : {:.6f} seconds".format(end - start))
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.1 and classID == 0:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.8,0.3)
    ind = []
    for i in range(0,len(classIDs)):
        if(classIDs[i]==0):
            ind.append(i)
    a = []
    b = []

    if len(idxs) > 0:
            for i in idxs.flatten():
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                a.append(x)
                b.append(y)
                
    distance=[] 
    nsd = []
    for i in range(0,len(a)-1):
        for k in range(1,len(a)):
            if(k==i):
                break
            else:
                x_dist = (a[k] - a[i])
                y_dist = (b[k] - b[i])
                d = math.sqrt(x_dist * x_dist + y_dist * y_dist)
                distance.append(d)
                if(d <=100):
                    nsd.append(i)
                    nsd.append(k)
                nsd = list(dict.fromkeys(nsd))
                print(nsd)
    color = (0, 0, 255) 
    for i in nsd:
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "Alert"
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
    color = (0, 255, 0) 
    if len(idxs) > 0:
        for i in idxs.flatten():
            if (i in nsd):
                break
            else:
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                text = 'OK'
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)   
    
    cv2.imshow("Social Distancing Detector", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    vid_writer.write(image)
vid_writer.release()
cap.release()
cv2.destroyAllWindows()    

Frame Prediction Time : 0.727057 seconds
[]
[0, 2]
[0, 2]
[0, 2, 4]
[0, 2, 4]
[0, 2, 4, 6]
[0, 2, 4, 6, 7]
[0, 2, 4, 6, 7, 8]
[0, 2, 4, 6, 7, 8]
[0, 2, 4, 6, 7, 8]
[0, 2, 4, 6, 7, 8]
[0, 2, 4, 6, 7, 8, 12]
[0, 2, 4, 6, 7, 8, 12, 13]
[0, 2, 4, 6, 7, 8, 12, 13]
[0, 2, 4, 6, 7, 8, 12, 13, 15]
[0, 2, 4, 6, 7, 8, 12, 13, 15]
[0, 2, 4, 6, 7, 8, 12, 13, 15]
[0, 2, 4, 6, 7, 8, 12, 13, 15]
[0, 2, 4, 6, 7, 8, 12, 13, 15]
[0, 2, 4, 6, 7, 8, 12, 13, 15]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 6, 7, 8, 12, 13, 15, 3]
[0, 2, 4, 

Frame Prediction Time : 0.627701 seconds
[]
[]
[0, 3]
[0, 3, 4]
[0, 3, 4]
[0, 3, 4]
[0, 3, 4]
[0, 3, 4]
[0, 3, 4]
[0, 3, 4, 10]
[0, 3, 4, 10, 11]
[0, 3, 4, 10, 11, 12]
[0, 3, 4, 10, 11, 12, 13]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5, 2, 6]
[0, 3, 4, 10, 11, 12, 13, 14, 5,

Frame Prediction Time : 0.564044 seconds
[0, 1]
[0, 1]
[0, 1]
[0, 1, 4]
[0, 1, 4]
[0, 1, 4]
[0, 1, 4, 7]
[0, 1, 4, 7]
[0, 1, 4, 7]
[0, 1, 4, 7, 10]
[0, 1, 4, 7, 10, 11]
[0, 1, 4, 7, 10, 11, 12]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13]
[0, 1, 4, 7, 10, 11, 12, 13, 8, 3]
[0, 1, 4, 7, 10, 11, 12, 13, 8, 3]
[0, 1, 4, 7

Frame Prediction Time : 0.481667 seconds
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1, 6]
[0, 1, 6]
[0, 1, 6, 8]
[0, 1, 6, 8, 9]
[0, 1, 6, 8, 9, 10]
[0, 1, 6, 8, 9, 10, 11]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10, 11, 12]
[0, 1, 6, 8, 9, 10,

Frame Prediction Time : 0.434799 seconds
[]
[0, 2]
[0, 2, 3]
[0, 2, 3]
[0, 2, 3]
[0, 2, 3, 6]
[0, 2, 3, 6]
[0, 2, 3, 6, 8]
[0, 2, 3, 6, 8]
[0, 2, 3, 6, 8, 10]
[0, 2, 3, 6, 8, 10]
[0, 2, 3, 6, 8, 10, 12]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 8, 10, 12, 13]
[0, 2, 3, 6, 

...

Frame Prediction Time : 0.446859 seconds
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[4, 1]
[4, 1]
[4, 1]
[4, 1, 5]
[4, 1, 5, 2]
[4, 1, 5, 2]
[4, 1, 5, 2]
[4, 1, 5, 2]
[4, 1, 5, 2, 6]
[4, 1, 5, 2, 6]
[4, 1, 5, 2, 6]
[4, 1, 5, 2, 6]
[4, 1, 5, 2, 6]
[4, 1, 5, 2, 6, 7]
[4, 1, 5, 2, 6, 7]
[4, 1, 5, 2, 6, 7]
[4, 1, 5, 2, 6, 7]
[4, 1, 5, 2, 6, 7]
[4, 1, 5, 2, 6, 7]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2, 6, 7, 8]
[4, 1, 5, 2,

Frame Prediction Time : 0.433801 seconds
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[2, 1]
[2, 1]
[2, 1]
[2, 1]
[2, 1]
[2, 1]
[2, 1]
[2, 1, 5]
[2, 1, 5, 3]
[2, 1, 5, 3]
[2, 1, 5, 3]
[2, 1, 5, 3]
[2, 1, 5, 3]
[2, 1, 5, 3]
[2, 1, 5, 3]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7]
[2, 1, 5, 3, 7, 10]
[2, 1, 5, 3, 7, 10, 4]
[2, 1, 5, 3, 7, 10, 4]
[2, 1, 5, 3, 7, 10, 4]
[2, 1, 5, 3, 7, 10, 4]
[2, 1, 5, 3, 7, 10, 4]
[2, 1, 5, 3, 7, 10, 4]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[2, 1, 5, 3, 7, 10, 4, 11]
[

Frame Prediction Time : 0.772902 seconds
[]
[]
[]
[]
[0, 5]
[0, 5]
[0, 5]
[0, 5, 8]
[0, 5, 8, 9]
[0, 5, 8, 9]
[0, 5, 8, 9]
[0, 5, 8, 9]
[0, 5, 8, 9, 13]
[0, 5, 8, 9, 13]
[0, 5, 8, 9, 13]
[0, 5, 8, 9, 13]
[0, 5, 8, 9, 13]
[0, 5, 8, 9, 13]
[0, 5, 8, 9, 13, 4, 2]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5, 8, 9, 13, 4, 2, 3, 6]
[0, 5,

Frame Prediction Time : 0.488682 seconds
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1, 8]
[0, 1, 8]
[0, 1, 8]
[0, 1, 8]
[0, 1, 8, 12]
[0, 1, 8, 12, 13]
[0, 1, 8, 12, 13, 14]
[0, 1, 8, 12, 13, 14]
[0, 1, 8, 12, 13, 14, 16]
[0, 1, 8, 12, 13, 14, 16]
[0, 1, 8, 12, 13, 14, 16]
[0, 1, 8, 12, 13, 14, 16]
[0, 1, 8, 12, 13, 14, 16]
[0, 1, 8, 12, 13, 14, 16]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8, 12, 13, 14, 16, 4, 3, 7]
[0, 1, 8,

Frame Prediction Time : 0.771967 seconds
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[3, 1]
[3, 1]
[3, 1]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9]
[3, 1, 4, 2, 7, 9, 10, 6]
[3, 1, 4, 2, 7, 9, 10, 6]
[3, 1, 4, 2, 7, 9, 10, 6]
[3, 1, 4, 2, 7, 9, 10, 6]
[3, 1, 4, 2, 7, 9, 10, 6, 11]
[3, 1, 4, 2, 7, 9, 10, 6, 11]
[3, 1, 4, 2, 7, 9, 10, 6, 11]
[3, 1, 4, 2, 7, 9, 10, 6, 11]
[3, 1, 4, 2, 7, 9, 10, 6, 11]
[3, 1, 4, 2, 7, 9, 10, 6, 11]
[3, 1, 4, 2, 7